# 🗺️ Exploring the nature of the `Title` field 🧑‍🔬
## As far as we know there is no way of extracting all the information in a _single pass algorithm_. We will assume a structure and drill down then modify the records that violate this structure.  Once we grok `Title`'s anatomy a cleaning script can be devised, but this stage is still exploratory.   
### The structure that many records follow is below
**PhotoAttribution** _ **Type-of-Hazardous-Site** _ **CommercialName** _ **CountyState** _ **MonthYear**

### Lets read in the data and get a sense for the its size

In [1]:
import pandas as pd
import re
import gc # releasing heavy dataframes

In [2]:
df = pd.read_csv('data/tidied_threaded_data_pull.csv') # get the file 
df.shape # get its dimensions and inspect in data viewer

(2718, 8)

## Give the `Title` column a quick browse

In [3]:
# Better to do in data viewer
# for rec in df['Title']: print(rec) 
# [rec for rec in df['Title']][0]

In [4]:
s = df.loc[0,'Title'] # grab the first cell's contents of the Title column
s.split('_'),s # demonstrate the split function

(['TAuch', 'FracSand-Mine-WisconsinProppants-Hixton', 'WI', 'June2019'],
 'TAuch_FracSand-Mine-WisconsinProppants-Hixton_WI_June2019')

# Column Creation: `Title_photo_attr`
Explore a the hyphen underscore notation

In [5]:
# Create a new column in the dataframe that contains the first part of the Title column
photo_attr = df.loc[:,'Title'].map(lambda x : x.split('_')[0]).copy(deep=True)
df['Title_photo_attr'] =  photo_attr
df['Title_photo_attr']=='TAuch'

0       True
1       True
2       True
3       True
4       True
        ... 
2713    True
2714    True
2715    True
2716    True
2717    True
Name: Title_photo_attr, Length: 2718, dtype: bool

In [6]:
all(df.loc[:,'Title_photo_attr']=='TAuch') # some not TED 

False

# 📷 Who took photos ?

In [7]:
photo_attribution = set(photographer for photographer in df['Title_photo_attr'])
photo_attribution, len(photo_attribution)

({'BHughes',
  'BLenker',
  'DJI',
  'DMartin',
  'DSC',
  'EJackson',
  'HBoschen',
  'KEdelstein',
  'LDaul',
  'MKelso',
  'MWhipple',
  'RJohnson',
  'RZarwell',
  'SRubright',
  'SSmith',
  'TAuch'},
 16)

In [8]:
sum(df['Title_photo_attr']=='TAuch') # how many are TED ?

2613

In [9]:
pic_taker_ct = dict()   # keys are the photographers, values are the counts
for photographer in photo_attribution:
    pic_taker_ct[photographer] = sum(df['Title_photo_attr']==photographer)  
sorted(pic_taker_ct.items(), key=lambda x : x[1])  # another great stat for the data report

[('DJI', 1),
 ('RZarwell', 1),
 ('MWhipple', 1),
 ('LDaul', 2),
 ('DSC', 2),
 ('HBoschen', 2),
 ('BHughes', 3),
 ('SRubright', 3),
 ('DMartin', 4),
 ('KEdelstein', 7),
 ('EJackson', 13),
 ('RJohnson', 14),
 ('BLenker', 14),
 ('MKelso', 14),
 ('SSmith', 24),
 ('TAuch', 2613)]

In [10]:
df['Title_photo_attr'].value_counts() # another way to get the counts

Title_photo_attr
TAuch         2613
SSmith          24
MKelso          14
BLenker         14
RJohnson        14
EJackson        13
KEdelstein       7
DMartin          4
BHughes          3
SRubright        3
HBoschen         2
LDaul            2
DSC              2
RZarwell         1
MWhipple         1
DJI              1
Name: count, dtype: int64

### Always try to prove to yourself that the results of an operation _make sense_ ༗

In [11]:
sum(pic_taker_ct.values()) == df.shape[0] # check that the counts add up    

True

# Column Creation: `title_split`
# Note the values "DJI" and "DSC" these break 💔 the schema. Lets find all of them and then remove 🗑️ them.

In [12]:
title_split = df.loc[:,'Title'].map(lambda x: x.split('_')).copy(deep=True)
df['title_split'] =  title_split

### the process of finding anomalous data can be very tedious...

In [13]:
# dji_or_dsc 
boolean_condition = df.loc[:, 'Title_photo_attr'].isin(['DJI','DSC'])
dirty_df = df[boolean_condition].copy(deep=True)    
dirty_df

,PhotoID,AlbumTitle,Title,Date_taken,Description,URL,Latitude,Longitude,Title_photo_attr,title_split
2588,53183405100,['Channels of Life: the Gulf Coast Buildout in...,DSC_1344_HighRes,2023-08-25 08:28:49,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/5318...,29.719927,-95.282273,DSC,"[DSC, 1344, HighRes]"
2592,53183405215,['Channels of Life: the Gulf Coast Buildout in...,DSC_1220_HighRes,2023-08-24 09:27:48,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/5318...,27.904619,-97.280487,DSC,"[DSC, 1220, HighRes]"
2649,53399929386,"['Appalachian Buildout', 'Environmental Justic...",DJI_0250_HighRes,2023-12-14 10:30:22,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/5339...,38.587983,-82.824592,DJI,"[DJI, 0250, HighRes]"


### Above are the three records way off the labeling schema

### Get links to the photos that Ted needs to fix
## Note...
## The first time I clicked on a link it return in error... but after refreshing the page once, it worked 🙃

In [14]:
for link in dirty_df['URL']: print(link)

https://www.flickr.com/photos/fractracker/53183405100/in/album-72157715839488573
https://www.flickr.com/photos/fractracker/53183405215/in/album-72157715839488573
https://www.flickr.com/photos/fractracker/53399929386/in/album-72157715916543893


In [15]:
# Remove the records from the original dataframe
df1 = df[~df['title_split'].str[0].isin(['DSC', 'DJI'])]
df1.shape

(2715, 10)

In [16]:
# recall the original dataframe dimensions
df.shape

(2718, 10)

#### the removed dataframe plus ➕ the new dataframe should equal 🟰 the original dataframe  

In [17]:
len(dirty_df)+len(df1)==len(df)

True

In [18]:
# all the data is in df1 so no use for df
del df
gc.collect()

17

### Lets continue to drill 🧐 down on `df1` the 'cleaner' 🧹 🛀 dataframe. At this stage we know each `Title` record begins with a photo attribution. Lets put the photo attribution in a new field with a simple `str.split('_')`

In [19]:
# demonstration of "join with slice." Join the elements after the first element in the list with an underscore
lst = ['DJI', '0250', 'HighRes']
lst[1:], '_'.join(lst[1:])

(['0250', 'HighRes'], '0250_HighRes')

In [20]:
df1.columns

Index(['PhotoID', 'AlbumTitle', 'Title', 'Date_taken', 'Description', 'URL',
       'Latitude', 'Longitude', 'Title_photo_attr', 'title_split'],
      dtype='object')

In [21]:
# Notice the last column is `title_split` (newly created columns are appended onto the end) 
# iloc reshresher, df.iloc[ which_rows, which_columns]
df1.iloc[:,-1] # "all rows of that last column"

0       [TAuch, FracSand-Mine-WisconsinProppants-Hixto...
1       [TAuch, FracSand-Mine-USSilica-Ottawa, IL, Lig...
2       [TAuch, FracSand-Mine-USSilica-Ottawa, IL, Lig...
3       [TAuch, FracSand-Mine-Unimin-NorthUtica, IL, L...
4       [TAuch, FracSand-Mine-Unimin-NorthUtica, IL, J...
                              ...                        
2713    [TAuch, Infrastructure-FracSand, RailSpur-Chie...
2714    [TAuch, Infrastructure-FracSand, RailSpur-Nort...
2715    [TAuch, Infrastructure-FracSand, Mine-HiCrush-...
2716    [TAuch, Infrastructure-FracSand, Mine-HiCrush-...
2717    [TAuch, Infrastructure-FracSand, Mine-HiCrush-...
Name: title_split, Length: 2715, dtype: object

**Aside**, occasionally the `exception pandas.errors.SettingWithCopyWarning` is raised. 'Warning raised when trying to set on a copied slice from a DataFrame' from [Documentation - Errors Module](https://pandas.pydata.org/docs/reference/api/pandas.errors.SettingWithCopyWarning.html#pandas-errors-settingwithcopywarning). Discussion on the warning, [Pandas User Guide](https://pandas.pydata.org/docs/user_guide/indexing.html#indexing-view-versus-copy), [Real Python](https://realpython.com/pandas-settingwithcopywarning/)
#### Crux of the issue is to know when you return a view or a copy of a dataframe
You can create a view or copy of a DataFrame with [.copy()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.copy.html). The parameter deep determines if you want a view (deep=False) or copy (deep=True)

In [22]:
# Create 'Title_no_photographer' field
no_pic_taker = df1.loc[:, 'title_split'].map(lambda x: '_'.join(x[1:])).copy(deep=True)   # make a copy of the series
df1.loc[:,'Title_no_photographer'] = no_pic_taker
df1.head()

,PhotoID,AlbumTitle,Title,Date_taken,Description,URL,Latitude,Longitude,Title_photo_attr,title_split,Title_no_photographer
0,49727911618,"['Frac Sand Mining', 'Wisconsin']",TAuch_FracSand-Mine-WisconsinProppants-Hixton_...,2019-06-05 14:07:05,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/4972...,44.376922,-91.069461,TAuch,"[TAuch, FracSand-Mine-WisconsinProppants-Hixto...",FracSand-Mine-WisconsinProppants-Hixton_WI_Jun...
1,49727914298,"['Frac Sand Mining', 'Illinois', 'Impoundment ...",TAuch_FracSand-Mine-USSilica-Ottawa_IL_LightHa...,2018-05-24 09:19:50,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/4972...,41.349366,-88.865747,TAuch,"[TAuch, FracSand-Mine-USSilica-Ottawa, IL, Lig...",FracSand-Mine-USSilica-Ottawa_IL_LightHawk_May...
2,49727914638,"['Frac Sand Mining', 'Frac Sand Transportation...",TAuch_FracSand-Mine-USSilica-Ottawa_IL_LightHa...,2016-06-24 10:47:44,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/4972...,41.349366,-88.865747,TAuch,"[TAuch, FracSand-Mine-USSilica-Ottawa, IL, Lig...",FracSand-Mine-USSilica-Ottawa_IL_LightHawk_Jun...
3,49727915018,"['Frac Sand Mining', 'Illinois', 'Impoundment ...",TAuch_FracSand-Mine-Unimin-NorthUtica_IL_Light...,2016-06-24 10:36:01,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/4972...,41.339217,-89.001553,TAuch,"[TAuch, FracSand-Mine-Unimin-NorthUtica, IL, L...",FracSand-Mine-Unimin-NorthUtica_IL_LightHawk_J...
4,49727915083,"['Frac Sand Mining', 'Illinois', 'Impoundment ...",TAuch_FracSand-Mine-Unimin-NorthUtica_IL_June2016,2016-06-24 12:13:10,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/4972...,41.351629,-88.992208,TAuch,"[TAuch, FracSand-Mine-Unimin-NorthUtica, IL, J...",FracSand-Mine-Unimin-NorthUtica_IL_June2016


### Let continue to drill down on the `Title_no_photographer` field. What general cleaning/extraction rules can we infer

In [23]:
for i in df1.loc[:, 'Title_no_photographer'] : print(i)

FracSand-Mine-WisconsinProppants-Hixton_WI_June2019
FracSand-Mine-USSilica-Ottawa_IL_LightHawk_May2018(1)
FracSand-Mine-USSilica-Ottawa_IL_LightHawk_June2016(2)
FracSand-Mine-Unimin-NorthUtica_IL_LightHawk_June2016(1)
FracSand-Mine-Unimin-NorthUtica_IL_June2016
FracSand-Mine-Unimin-NorthUtica_IL_June2016(4)
FracSand-Mine-Unimin-NorthUtica_IL_June2016(2)
FracSand-Mine-Unimin-NorthUtica_IL_Aug2017
FracSand-Mine-Unimin-LaSalleCounty_IL_June2016
FracSand-Mine-Unimin_TechniSand-TroyGrove_IL_May2018
FracSand-Mine-Unimin_TechniSand-TroyGrove_IL_June2016
FracSand-Mine-NorthernWhiteSands-Ottawa_IL_May2018
FracSand-Mine-NorthernWhiteSands-Ottawa_IL_LightHawk_May2018(1)
FracSand-Mine-NorthernWhiteSands-Ottawa_IL_June2016(1)
FracSand-Mine-FairmountSantrol-Wedron_IL_May2018(3)
FracSand-Mine-FairmountSantrol-Wedron_IL_June2016(3)
FracSand-Mine-SylvanianMinerals-SouthRockwood_MI_July2015
FracSand-Mine-SylvanianMinerals-SouthRockwood_MI_Aug2017(1)
FracSand-Mine-SargentSand-Ludington_MI_Sept2014
FracSa

### lets ask the question how many records do not end in a 4 digit year ? 

In [24]:
manual_clean = list()
for i in df1.loc[:, 'Title_no_photographer'] : 
    # [\d]{4}$       4 digits start from the end of the string
    pat = r"([\d]{4})$"
    match = re.search(pat, i)
    if not match: # what records do NOT match my pattern? 
        manual_clean.append(i) # dave what does NOT match what i expected
        print(i)
        print('----------------')

FracSand-Mine-USSilica-Ottawa_IL_LightHawk_May2018(1)
----------------
FracSand-Mine-USSilica-Ottawa_IL_LightHawk_June2016(2)
----------------
FracSand-Mine-Unimin-NorthUtica_IL_LightHawk_June2016(1)
----------------
FracSand-Mine-Unimin-NorthUtica_IL_June2016(4)
----------------
FracSand-Mine-Unimin-NorthUtica_IL_June2016(2)
----------------
FracSand-Mine-NorthernWhiteSands-Ottawa_IL_LightHawk_May2018(1)
----------------
FracSand-Mine-NorthernWhiteSands-Ottawa_IL_June2016(1)
----------------
FracSand-Mine-FairmountSantrol-Wedron_IL_May2018(3)
----------------
FracSand-Mine-FairmountSantrol-Wedron_IL_June2016(3)
----------------
FracSand-Mine-SylvanianMinerals-SouthRockwood_MI_Aug2017(1)
----------------
FracSand-Mine-SargentSand-Ludington_MI_Oct2016(4)
----------------
FracSand-Mine-SargentSand-Ludington_MI_Oct2016(3)
----------------
FracSand-Mine-RosyMound-GrandHaven_MI_Oct2016(11)
----------------
FracSand-Mine-RosyMound-GrandHaven_MI_Oct2016(10)
----------------
FracSand-Mine-Rosy

## Lets handle the `(n)` case as well

In [25]:
manual_clean = list()
for i in df1.loc[:, 'Title_no_photographer'] : 
    # [\d]{4}       4 digits
    # [)]           a parenthesis
    # [\d]{1,2}      1 or 2 digits 
    # [(]           a  parenthesis
    ### optional or once above three lines
    pat = r"([\d]{4})([(][\d]{1,2}[)])?$"
    match = re.search(pat, i)
    if not match: # what records do NOT match my pattern? 
        manual_clean.append(i) # dave what does NOT match what i expected
        print(i)
        print('----------------')

Infrastructure-Frackpad_Flaring-EagleFord-Esseville-TX_Nov2019.jpg
----------------
Infrastructure-CompressorStation-TransCanada-KalkaskaCounty_MI_May2016.jpg(1)
----------------
Infrastructure-CompressorStation-TransCanada-KalkaskaCounty_MI_May2016.jpg
----------------
Transportation-InjectionWell_Wastewater_BrineHauler-AshtabulaCounty_OH_Nov2015.jpg
----------------


### Interesting! Some title fields end with `.jpg` .... 🤔
### Lets manually clean these records 

In [26]:
manual_clean

['Infrastructure-Frackpad_Flaring-EagleFord-Esseville-TX_Nov2019.jpg',
 'Infrastructure-CompressorStation-TransCanada-KalkaskaCounty_MI_May2016.jpg(1)',
 'Infrastructure-CompressorStation-TransCanada-KalkaskaCounty_MI_May2016.jpg',
 'Transportation-InjectionWell_Wastewater_BrineHauler-AshtabulaCounty_OH_Nov2015.jpg']

In [27]:
# strategy will be to split on period but first let check if there are any periods elsewhere
'.' in manual_clean[0]

True

In [28]:
# checking for periods are elsewhere
for rec in df1['Title_no_photographer']: 
    if '.' in rec:
        print(rec)

Infrastructure-Frackpad_Flaring-EagleFord-Esseville-TX_Nov2019.jpg
Infrastructure-CompressorStation-TransCanada-KalkaskaCounty_MI_May2016.jpg(1)
Infrastructure-CompressorStation-TransCanada-KalkaskaCounty_MI_May2016.jpg
Transportation-InjectionWell_Wastewater_BrineHauler-AshtabulaCounty_OH_Nov2015.jpg


### Periods are only in the records found by the previous regular expression

In [29]:
manual_clean == [rec for rec in df1['Title_no_photographer'] if '.' in rec] # verify it is safe to remove split on period

True

In [30]:
def trimFileExt(x):
    if x in manual_clean: return x.split('.')[0]
    return x 
trimmedRecs = df1.loc[:, "Title_no_photographer"].apply(trimFileExt).copy(deep=True)
df1.loc[:, "Title_no_photographer"] = trimmedRecs
df1

,PhotoID,AlbumTitle,Title,Date_taken,Description,URL,Latitude,Longitude,Title_photo_attr,title_split,Title_no_photographer
0,49727911618,"['Frac Sand Mining', 'Wisconsin']",TAuch_FracSand-Mine-WisconsinProppants-Hixton_...,2019-06-05 14:07:05,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/4972...,44.376922,-91.069461,TAuch,"[TAuch, FracSand-Mine-WisconsinProppants-Hixto...",FracSand-Mine-WisconsinProppants-Hixton_WI_Jun...
1,49727914298,"['Frac Sand Mining', 'Illinois', 'Impoundment ...",TAuch_FracSand-Mine-USSilica-Ottawa_IL_LightHa...,2018-05-24 09:19:50,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/4972...,41.349366,-88.865747,TAuch,"[TAuch, FracSand-Mine-USSilica-Ottawa, IL, Lig...",FracSand-Mine-USSilica-Ottawa_IL_LightHawk_May...
2,49727914638,"['Frac Sand Mining', 'Frac Sand Transportation...",TAuch_FracSand-Mine-USSilica-Ottawa_IL_LightHa...,2016-06-24 10:47:44,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/4972...,41.349366,-88.865747,TAuch,"[TAuch, FracSand-Mine-USSilica-Ottawa, IL, Lig...",FracSand-Mine-USSilica-Ottawa_IL_LightHawk_Jun...
3,49727915018,"['Frac Sand Mining', 'Illinois', 'Impoundment ...",TAuch_FracSand-Mine-Unimin-NorthUtica_IL_Light...,2016-06-24 10:36:01,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/4972...,41.339217,-89.001553,TAuch,"[TAuch, FracSand-Mine-Unimin-NorthUtica, IL, L...",FracSand-Mine-Unimin-NorthUtica_IL_LightHawk_J...
4,49727915083,"['Frac Sand Mining', 'Illinois', 'Impoundment ...",TAuch_FracSand-Mine-Unimin-NorthUtica_IL_June2016,2016-06-24 12:13:10,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/4972...,41.351629,-88.992208,TAuch,"[TAuch, FracSand-Mine-Unimin-NorthUtica, IL, J...",FracSand-Mine-Unimin-NorthUtica_IL_June2016
...,...,...,...,...,...,...,...,...,...,...,...
2713,53706158340,"['Communities', 'Culture & Livelihoods', 'Envi...",TAuch_Infrastructure-FracSand_RailSpur-Chiefta...,2024-05-03 15:37:18,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/5370...,45.257286,-91.609020,TAuch,"[TAuch, Infrastructure-FracSand, RailSpur-Chie...",Infrastructure-FracSand_RailSpur-ChieftainSand...
2714,53706158375,"['Communities', 'Culture & Livelihoods', 'Envi...",TAuch_Infrastructure-FracSand_RailSpur-Norther...,2024-05-03 15:24:13,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/5370...,45.281691,-91.626303,TAuch,"[TAuch, Infrastructure-FracSand, RailSpur-Nort...",Infrastructure-FracSand_RailSpur-NorthernIndus...
2715,53706158545,"['Communities', 'Culture & Livelihoods', 'Envi...",TAuch_Infrastructure-FracSand_Mine-HiCrush-Tre...,2024-05-05 11:05:59,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/5370...,44.347402,-91.360048,TAuch,"[TAuch, Infrastructure-FracSand, Mine-HiCrush-...",Infrastructure-FracSand_Mine-HiCrush-Trempeale...
2716,53706158630,"['Communities', 'Culture & Livelihoods', 'Envi...",TAuch_Infrastructure-FracSand_Mine-HiCrush-Tre...,2024-05-05 11:03:04,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/5370...,44.354636,-91.362434,TAuch,"[TAuch, Infrastructure-FracSand, Mine-HiCrush-...",Infrastructure-FracSand_Mine-HiCrush-Trempeale...


### Removed and lets look again

In [31]:
manual_clean = list()
for i in df1.loc[:, 'Title_no_photographer'] : 
    pat = r"([\d]{4})([(][\d]{1,2}[)])?$"
    match = re.search(pat, i)
    if not match: # what records do NOT match my pattern? 
        manual_clean.append(i) # dave what does NOT match what i expected
        print(i)
        print('----------------')

## the above yields no results! success! 👏 All records have a trailing NNNN or (N) where is N is a digit [0-9]
## Now remove those pesky `(n)` 🦟

In [32]:
manual_clean = list()
for i in df1.loc[:, 'Title_no_photographer'] : 
    # [\d]{4}       4 digits
    pat = r"([\d]{4})$"
    match = re.search(pat, i)
    if not match: # what records do NOT match my pattern? 
        manual_clean.append(i) # dave what does NOT match what i expected
        print(i)
        print('----------------')

FracSand-Mine-USSilica-Ottawa_IL_LightHawk_May2018(1)
----------------
FracSand-Mine-USSilica-Ottawa_IL_LightHawk_June2016(2)
----------------
FracSand-Mine-Unimin-NorthUtica_IL_LightHawk_June2016(1)
----------------
FracSand-Mine-Unimin-NorthUtica_IL_June2016(4)
----------------
FracSand-Mine-Unimin-NorthUtica_IL_June2016(2)
----------------
FracSand-Mine-NorthernWhiteSands-Ottawa_IL_LightHawk_May2018(1)
----------------
FracSand-Mine-NorthernWhiteSands-Ottawa_IL_June2016(1)
----------------
FracSand-Mine-FairmountSantrol-Wedron_IL_May2018(3)
----------------
FracSand-Mine-FairmountSantrol-Wedron_IL_June2016(3)
----------------
FracSand-Mine-SylvanianMinerals-SouthRockwood_MI_Aug2017(1)
----------------
FracSand-Mine-SargentSand-Ludington_MI_Oct2016(4)
----------------
FracSand-Mine-SargentSand-Ludington_MI_Oct2016(3)
----------------
FracSand-Mine-RosyMound-GrandHaven_MI_Oct2016(11)
----------------
FracSand-Mine-RosyMound-GrandHaven_MI_Oct2016(10)
----------------
FracSand-Mine-Rosy

### Developing a strategy...

In [33]:
# demonstrate split on open parenthesis
'FracSand-Mine-USSilica-Ottawa_IL_LightHawk_May2018(1)'.split('(')

['FracSand-Mine-USSilica-Ottawa_IL_LightHawk_May2018', '1)']

In [34]:
# demonstrate count open parenthesis
'FracSand-Mine-USSilica-Ottawa_IL_LightHawk_May2018(1)'.count('(') 

1

In [35]:
# verify there is only one opening parenthesis
test_man_clean_list = [record for record in manual_clean if record.count('(') != 1]
test_man_clean_list

[]

In [36]:
# Are there parenthesis elsewhere in the column that would infer with this strategy?
paren_recs = list()
for i in df1.loc[:, 'Title_no_photographer'] : 
    if '(' in i:
        paren_recs.append(i)

paren_recs == manual_clean

True

## The above comparison asserts the only records that have parenthesis are the one that were previous found in the manual_clean list.

In [37]:
def trimParen(x):
    if x in manual_clean: return x.split('(')[0]
    return x 
trimmed = df1.loc[:, "Title_no_photographer"].apply(trimParen).copy()
df1.loc[:, "Title_no_photographer"] = trimmed 

In [38]:
# Verify trimming worked
manual_clean = list()
for i in df1.loc[:, 'Title_no_photographer'] : 
    # [\d]{4}       4 digits
    pat = r"([\d]{4})$"
    match = re.search(pat, i)
    if not match: # what records do NOT match my pattern? 
        manual_clean.append(i) # dave what does NOT match what i expected
        print(i)
        print('----------------')
manual_clean

[]

In [39]:
for i in df1.loc[:, 'Title_no_photographer'] : print(i)

FracSand-Mine-WisconsinProppants-Hixton_WI_June2019
FracSand-Mine-USSilica-Ottawa_IL_LightHawk_May2018
FracSand-Mine-USSilica-Ottawa_IL_LightHawk_June2016
FracSand-Mine-Unimin-NorthUtica_IL_LightHawk_June2016
FracSand-Mine-Unimin-NorthUtica_IL_June2016
FracSand-Mine-Unimin-NorthUtica_IL_June2016
FracSand-Mine-Unimin-NorthUtica_IL_June2016
FracSand-Mine-Unimin-NorthUtica_IL_Aug2017
FracSand-Mine-Unimin-LaSalleCounty_IL_June2016
FracSand-Mine-Unimin_TechniSand-TroyGrove_IL_May2018
FracSand-Mine-Unimin_TechniSand-TroyGrove_IL_June2016
FracSand-Mine-NorthernWhiteSands-Ottawa_IL_May2018
FracSand-Mine-NorthernWhiteSands-Ottawa_IL_LightHawk_May2018
FracSand-Mine-NorthernWhiteSands-Ottawa_IL_June2016
FracSand-Mine-FairmountSantrol-Wedron_IL_May2018
FracSand-Mine-FairmountSantrol-Wedron_IL_June2016
FracSand-Mine-SylvanianMinerals-SouthRockwood_MI_July2015
FracSand-Mine-SylvanianMinerals-SouthRockwood_MI_Aug2017
FracSand-Mine-SargentSand-Ludington_MI_Sept2014
FracSand-Mine-SargentSand-Ludington_

In [40]:
# sanity check! do i still have all my records ?
df1.shape

(2715, 11)

In [41]:
# At this stage we know FOR SURE each `Title_no_photographer` field ends in a 4 digit year.
# Lets extract the year first and then the month. 
for i in df1.loc[:, 'Title_no_photographer'] : print(i[:-4])

FracSand-Mine-WisconsinProppants-Hixton_WI_June
FracSand-Mine-USSilica-Ottawa_IL_LightHawk_May
FracSand-Mine-USSilica-Ottawa_IL_LightHawk_June
FracSand-Mine-Unimin-NorthUtica_IL_LightHawk_June
FracSand-Mine-Unimin-NorthUtica_IL_June
FracSand-Mine-Unimin-NorthUtica_IL_June
FracSand-Mine-Unimin-NorthUtica_IL_June
FracSand-Mine-Unimin-NorthUtica_IL_Aug
FracSand-Mine-Unimin-LaSalleCounty_IL_June
FracSand-Mine-Unimin_TechniSand-TroyGrove_IL_May
FracSand-Mine-Unimin_TechniSand-TroyGrove_IL_June
FracSand-Mine-NorthernWhiteSands-Ottawa_IL_May
FracSand-Mine-NorthernWhiteSands-Ottawa_IL_LightHawk_May
FracSand-Mine-NorthernWhiteSands-Ottawa_IL_June
FracSand-Mine-FairmountSantrol-Wedron_IL_May
FracSand-Mine-FairmountSantrol-Wedron_IL_June
FracSand-Mine-SylvanianMinerals-SouthRockwood_MI_July
FracSand-Mine-SylvanianMinerals-SouthRockwood_MI_Aug
FracSand-Mine-SargentSand-Ludington_MI_Sept
FracSand-Mine-SargentSand-Ludington_MI_Oct
FracSand-Mine-SargentSand-Ludington_MI_Oct
FracSand-Mine-RosyMound-Gr

In [42]:
for i in df1.loc[:, 'Title_no_photographer'] : print(i[-4:])

2019
2018
2016
2016
2016
2016
2016
2017
2016
2018
2016
2018
2018
2016
2018
2016
2015
2017
2014
2016
2016
2016
2016
2016
2016
2016
2016
2017
2017
2015
2019
2019
2019
2019
2019
2013
2013
2013
2013
2016
2016
2016
2015
2017
2016
2016
2016
2015
2015
2015
2016
2013
2013
2016
2016
2019
2018
2018
2018
2016
2016
2016
2018
2018
2018
2018
2018
2016
2016
2016
2015
2014
2016
2016
2016
2016
2016
2014
2017
2019
2019
2019
2017
2013
2013
2013
2013
2017
2013
2017
2016
2016
2016
2016
2018
2015
2015
2015
2017
2015
2013
2016
2013
2018
2018
2015
2015
2014
2015
2015
2016
2016
2013
2016
2017
2016
2018
2016
2018
2016
2016
2017
2018
2018
2018
2016
2018
2016
2016
2017
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2014
2015
2019
2019
2019
2019
2019
2013
2017
2017
2016
2013
2013
2016
2016
2016
2016
2016
2015
2016
2017
2016
2014
2014
2014
2019
2019
2019
2019
2019
2019
2016
2016
2017
2015
2018
2018
2019
2019
2019
2019
2019
2017
2018
2016
2017
2015
2015
2018
2018
2019
2019
2019
2019
2019
2019


In [43]:
yrs = dict()
for i in df1.loc[:, 'Title_no_photographer'] : 
    yr = i[-4:]
    if yr in yrs: yrs[yr] += 1
    else: yrs[yr] = 1   
yrs

{'2019': 518,
 '2018': 332,
 '2016': 182,
 '2017': 122,
 '2015': 103,
 '2014': 10,
 '2013': 32,
 '2020': 197,
 '2021': 503,
 '2022': 210,
 '2023': 449,
 '2024': 57}

### The above dictionary verifies the only characters that were extracted are digits! This is due to the mathematical set properties of the python `dict` keys 

In [44]:
just_years = df1.loc[:, 'Title_no_photographer'].map(lambda x: x[-4:]).copy(deep=True)
df1['Title_yr'] = just_years
df1

,PhotoID,AlbumTitle,Title,Date_taken,Description,URL,Latitude,Longitude,Title_photo_attr,title_split,Title_no_photographer,Title_yr
0,49727911618,"['Frac Sand Mining', 'Wisconsin']",TAuch_FracSand-Mine-WisconsinProppants-Hixton_...,2019-06-05 14:07:05,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/4972...,44.376922,-91.069461,TAuch,"[TAuch, FracSand-Mine-WisconsinProppants-Hixto...",FracSand-Mine-WisconsinProppants-Hixton_WI_Jun...,2019
1,49727914298,"['Frac Sand Mining', 'Illinois', 'Impoundment ...",TAuch_FracSand-Mine-USSilica-Ottawa_IL_LightHa...,2018-05-24 09:19:50,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/4972...,41.349366,-88.865747,TAuch,"[TAuch, FracSand-Mine-USSilica-Ottawa, IL, Lig...",FracSand-Mine-USSilica-Ottawa_IL_LightHawk_May...,2018
2,49727914638,"['Frac Sand Mining', 'Frac Sand Transportation...",TAuch_FracSand-Mine-USSilica-Ottawa_IL_LightHa...,2016-06-24 10:47:44,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/4972...,41.349366,-88.865747,TAuch,"[TAuch, FracSand-Mine-USSilica-Ottawa, IL, Lig...",FracSand-Mine-USSilica-Ottawa_IL_LightHawk_Jun...,2016
3,49727915018,"['Frac Sand Mining', 'Illinois', 'Impoundment ...",TAuch_FracSand-Mine-Unimin-NorthUtica_IL_Light...,2016-06-24 10:36:01,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/4972...,41.339217,-89.001553,TAuch,"[TAuch, FracSand-Mine-Unimin-NorthUtica, IL, L...",FracSand-Mine-Unimin-NorthUtica_IL_LightHawk_J...,2016
4,49727915083,"['Frac Sand Mining', 'Illinois', 'Impoundment ...",TAuch_FracSand-Mine-Unimin-NorthUtica_IL_June2016,2016-06-24 12:13:10,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/4972...,41.351629,-88.992208,TAuch,"[TAuch, FracSand-Mine-Unimin-NorthUtica, IL, J...",FracSand-Mine-Unimin-NorthUtica_IL_June2016,2016
...,...,...,...,...,...,...,...,...,...,...,...,...
2713,53706158340,"['Communities', 'Culture & Livelihoods', 'Envi...",TAuch_Infrastructure-FracSand_RailSpur-Chiefta...,2024-05-03 15:37:18,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/5370...,45.257286,-91.609020,TAuch,"[TAuch, Infrastructure-FracSand, RailSpur-Chie...",Infrastructure-FracSand_RailSpur-ChieftainSand...,2024
2714,53706158375,"['Communities', 'Culture & Livelihoods', 'Envi...",TAuch_Infrastructure-FracSand_RailSpur-Norther...,2024-05-03 15:24:13,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/5370...,45.281691,-91.626303,TAuch,"[TAuch, Infrastructure-FracSand, RailSpur-Nort...",Infrastructure-FracSand_RailSpur-NorthernIndus...,2024
2715,53706158545,"['Communities', 'Culture & Livelihoods', 'Envi...",TAuch_Infrastructure-FracSand_Mine-HiCrush-Tre...,2024-05-05 11:05:59,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/5370...,44.347402,-91.360048,TAuch,"[TAuch, Infrastructure-FracSand, Mine-HiCrush-...",Infrastructure-FracSand_Mine-HiCrush-Trempeale...,2024
2716,53706158630,"['Communities', 'Culture & Livelihoods', 'Envi...",TAuch_Infrastructure-FracSand_Mine-HiCrush-Tre...,2024-05-05 11:03:04,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/5370...,44.354636,-91.362434,TAuch,"[TAuch, Infrastructure-FracSand, Mine-HiCrush-...",Infrastructure-FracSand_Mine-HiCrush-Trempeale...,2024


In [45]:
# Notice the switch of the colon in the slice. Now lets get everything but the last 4 characters
minus_yrs = df1.loc[:, "Title_no_photographer"].apply(lambda x: x[:-4]).copy(deep=True)
df1["Title_no_photographer_or_yr"] = minus_yrs
df1

,PhotoID,AlbumTitle,Title,Date_taken,Description,URL,Latitude,Longitude,Title_photo_attr,title_split,Title_no_photographer,Title_yr,Title_no_photographer_or_yr
0,49727911618,"['Frac Sand Mining', 'Wisconsin']",TAuch_FracSand-Mine-WisconsinProppants-Hixton_...,2019-06-05 14:07:05,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/4972...,44.376922,-91.069461,TAuch,"[TAuch, FracSand-Mine-WisconsinProppants-Hixto...",FracSand-Mine-WisconsinProppants-Hixton_WI_Jun...,2019,FracSand-Mine-WisconsinProppants-Hixton_WI_June
1,49727914298,"['Frac Sand Mining', 'Illinois', 'Impoundment ...",TAuch_FracSand-Mine-USSilica-Ottawa_IL_LightHa...,2018-05-24 09:19:50,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/4972...,41.349366,-88.865747,TAuch,"[TAuch, FracSand-Mine-USSilica-Ottawa, IL, Lig...",FracSand-Mine-USSilica-Ottawa_IL_LightHawk_May...,2018,FracSand-Mine-USSilica-Ottawa_IL_LightHawk_May
2,49727914638,"['Frac Sand Mining', 'Frac Sand Transportation...",TAuch_FracSand-Mine-USSilica-Ottawa_IL_LightHa...,2016-06-24 10:47:44,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/4972...,41.349366,-88.865747,TAuch,"[TAuch, FracSand-Mine-USSilica-Ottawa, IL, Lig...",FracSand-Mine-USSilica-Ottawa_IL_LightHawk_Jun...,2016,FracSand-Mine-USSilica-Ottawa_IL_LightHawk_June
3,49727915018,"['Frac Sand Mining', 'Illinois', 'Impoundment ...",TAuch_FracSand-Mine-Unimin-NorthUtica_IL_Light...,2016-06-24 10:36:01,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/4972...,41.339217,-89.001553,TAuch,"[TAuch, FracSand-Mine-Unimin-NorthUtica, IL, L...",FracSand-Mine-Unimin-NorthUtica_IL_LightHawk_J...,2016,FracSand-Mine-Unimin-NorthUtica_IL_LightHawk_June
4,49727915083,"['Frac Sand Mining', 'Illinois', 'Impoundment ...",TAuch_FracSand-Mine-Unimin-NorthUtica_IL_June2016,2016-06-24 12:13:10,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/4972...,41.351629,-88.992208,TAuch,"[TAuch, FracSand-Mine-Unimin-NorthUtica, IL, J...",FracSand-Mine-Unimin-NorthUtica_IL_June2016,2016,FracSand-Mine-Unimin-NorthUtica_IL_June
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2713,53706158340,"['Communities', 'Culture & Livelihoods', 'Envi...",TAuch_Infrastructure-FracSand_RailSpur-Chiefta...,2024-05-03 15:37:18,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/5370...,45.257286,-91.609020,TAuch,"[TAuch, Infrastructure-FracSand, RailSpur-Chie...",Infrastructure-FracSand_RailSpur-ChieftainSand...,2024,Infrastructure-FracSand_RailSpur-ChieftainSand...
2714,53706158375,"['Communities', 'Culture & Livelihoods', 'Envi...",TAuch_Infrastructure-FracSand_RailSpur-Norther...,2024-05-03 15:24:13,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/5370...,45.281691,-91.626303,TAuch,"[TAuch, Infrastructure-FracSand, RailSpur-Nort...",Infrastructure-FracSand_RailSpur-NorthernIndus...,2024,Infrastructure-FracSand_RailSpur-NorthernIndus...
2715,53706158545,"['Communities', 'Culture & Livelihoods', 'Envi...",TAuch_Infrastructure-FracSand_Mine-HiCrush-Tre...,2024-05-05 11:05:59,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/5370...,44.347402,-91.360048,TAuch,"[TAuch, Infrastructure-FracSand, Mine-HiCrush-...",Infrastructure-FracSand_Mine-HiCrush-Trempeale...,2024,Infrastructure-FracSand_Mine-HiCrush-Trempeale...
2716,53706158630,"['Communities', 'Culture & Livelihoods', 'Envi...",TAuch_Infrastructure-FracSand_Mine-HiCrush-Tre...,2024-05-05 11:03:04,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/5370...,44.354636,-91.362434,TAuch,"[TAuch, Infrastructure-FracSand, Mine-HiCrush-...",Infrastructure-FracSand_Mine-HiCrush-Trempeale...,2024,Infrastructure-FracSand_Mine-HiCrush-Trempeale...


In [46]:
# lets pick off the month/season
month_season = set()
for i in df1.loc[:, 'Title_no_photographer_or_yr'] : 
    ii = i.split('_')[-1]
    month_season.add(ii)
month_season

{'',
 'Apr',
 'April',
 'Aug',
 'August',
 'Dec',
 'December',
 'Fall',
 'Feb',
 'February',
 'Jan',
 'July',
 'Jun',
 'June',
 'March',
 'May',
 'Nov',
 'November',
 'Oct',
 'October',
 'Sept',
 'Summer',
 'TX-Nov'}

In [47]:
# check occurrences of each month/season
month_season_d = dict()
for i in df1.loc[:, 'Title_no_photographer_or_yr'] : 
    ii = i.split('_')[-1]
    if ii in month_season_d: month_season_d[ii] += 1
    else: month_season_d[ii] = 1
month_season_d

{'June': 257,
 'May': 300,
 'Aug': 212,
 'July': 166,
 'Sept': 291,
 'Oct': 118,
 'Nov': 213,
 'Fall': 17,
 'Apr': 19,
 '': 4,
 'Dec': 21,
 'Jan': 28,
 'March': 140,
 'November': 1,
 'April': 177,
 'TX-Nov': 1,
 'Feb': 41,
 'August': 393,
 'October': 87,
 'Jun': 32,
 'December': 144,
 'Summer': 4,
 'February': 49}

### notice the empty string and the 'TX-Nov' in the set above. These cases need to be handled separately

In [48]:
muddy = list()
for idx,row in df1.iterrows():
    ii = row['Title_no_photographer_or_yr'].split('_')[-1] 
    if ii == '' or ii == 'TX-Nov':
        muddy.append(idx)
muddy

[51, 52, 100, 102, 614]

In [49]:
df1.loc[muddy]

,PhotoID,AlbumTitle,Title,Date_taken,Description,URL,Latitude,Longitude,Title_photo_attr,title_split,Title_no_photographer,Title_yr,Title_no_photographer_or_yr
51,49728093488,"['Frac Sand Storage & Processing', 'Frac Sand ...",TAuch_FracSand-Transport-GreatNorthernSand-Dov...,2020-04-02 13:10:29,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/4972...,45.256461,-91.600227,TAuch,"[TAuch, FracSand-Transport-GreatNorthernSand-D...",FracSand-Transport-GreatNorthernSand-Dovre-WI_...,2013,FracSand-Transport-GreatNorthernSand-Dovre-WI_
52,49728093538,"['Frac Sand Storage & Processing', 'Frac Sand ...",TAuch_FracSand-Transport-ChippewaSand-CooksVal...,2020-04-02 13:10:30,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/4972...,45.200122,-91.561346,TAuch,"[TAuch, FracSand-Transport-ChippewaSand-CooksV...",FracSand-Transport-ChippewaSand-CooksValley-WI...,2013,FracSand-Transport-ChippewaSand-CooksValley-WI_
100,49728600276,"['Frac Sand Storage & Processing', 'Wisconsin']",TAuch_FracSand-Mine-SuperiorSilicaSands-NewAub...,2020-04-02 12:57:28,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/4972...,45.210554,-91.565766,TAuch,"[TAuch, FracSand-Mine-SuperiorSilicaSands-NewA...",FracSand-Mine-SuperiorSilicaSands-NewAuburn-WI...,2013,FracSand-Mine-SuperiorSilicaSands-NewAuburn-WI_
102,49728601571,"['Frac Sand Storage & Processing', 'Frac Sand ...",TAuch_FracSand-Mine-ChieftainSand-Dovre-WI_201...,2020-04-02 12:57:53,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/4972...,45.253591,-91.598167,TAuch,"[TAuch, FracSand-Mine-ChieftainSand-Dovre-WI, ...",FracSand-Mine-ChieftainSand-Dovre-WI_2013,2013,FracSand-Mine-ChieftainSand-Dovre-WI_
614,49785865077,['Channels of Life: the Gulf Coast Buildout in...,TAuch_Infrastructure-InjectionWell-TrisunEnerg...,2019-11-14 13:16:57,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/4978...,28.455109,-98.087847,TAuch,"[TAuch, Infrastructure-InjectionWell-TrisunEne...",Infrastructure-InjectionWell-TrisunEnergyServi...,2019,Infrastructure-InjectionWell-TrisunEnergyServi...


In [50]:
# pandas is all about no loops, so lets do this without a loop
cond = df1['Title_no_photographer_or_yr'].apply(lambda x :x.split('_')[-1]=='' or x.split('_')[-1] == 'TX-Nov')
matching_records = df1.loc[cond].copy(deep=True)
matching_records

,PhotoID,AlbumTitle,Title,Date_taken,Description,URL,Latitude,Longitude,Title_photo_attr,title_split,Title_no_photographer,Title_yr,Title_no_photographer_or_yr
51,49728093488,"['Frac Sand Storage & Processing', 'Frac Sand ...",TAuch_FracSand-Transport-GreatNorthernSand-Dov...,2020-04-02 13:10:29,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/4972...,45.256461,-91.600227,TAuch,"[TAuch, FracSand-Transport-GreatNorthernSand-D...",FracSand-Transport-GreatNorthernSand-Dovre-WI_...,2013,FracSand-Transport-GreatNorthernSand-Dovre-WI_
52,49728093538,"['Frac Sand Storage & Processing', 'Frac Sand ...",TAuch_FracSand-Transport-ChippewaSand-CooksVal...,2020-04-02 13:10:30,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/4972...,45.200122,-91.561346,TAuch,"[TAuch, FracSand-Transport-ChippewaSand-CooksV...",FracSand-Transport-ChippewaSand-CooksValley-WI...,2013,FracSand-Transport-ChippewaSand-CooksValley-WI_
100,49728600276,"['Frac Sand Storage & Processing', 'Wisconsin']",TAuch_FracSand-Mine-SuperiorSilicaSands-NewAub...,2020-04-02 12:57:28,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/4972...,45.210554,-91.565766,TAuch,"[TAuch, FracSand-Mine-SuperiorSilicaSands-NewA...",FracSand-Mine-SuperiorSilicaSands-NewAuburn-WI...,2013,FracSand-Mine-SuperiorSilicaSands-NewAuburn-WI_
102,49728601571,"['Frac Sand Storage & Processing', 'Frac Sand ...",TAuch_FracSand-Mine-ChieftainSand-Dovre-WI_201...,2020-04-02 12:57:53,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/4972...,45.253591,-91.598167,TAuch,"[TAuch, FracSand-Mine-ChieftainSand-Dovre-WI, ...",FracSand-Mine-ChieftainSand-Dovre-WI_2013,2013,FracSand-Mine-ChieftainSand-Dovre-WI_
614,49785865077,['Channels of Life: the Gulf Coast Buildout in...,TAuch_Infrastructure-InjectionWell-TrisunEnerg...,2019-11-14 13:16:57,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/4978...,28.455109,-98.087847,TAuch,"[TAuch, Infrastructure-InjectionWell-TrisunEne...",Infrastructure-InjectionWell-TrisunEnergyServi...,2019,Infrastructure-InjectionWell-TrisunEnergyServi...


In [51]:
# remind yourself what the average looks like, use the data viewer
for i in df1['Title_no_photographer_or_yr']: print(i)

FracSand-Mine-WisconsinProppants-Hixton_WI_June
FracSand-Mine-USSilica-Ottawa_IL_LightHawk_May
FracSand-Mine-USSilica-Ottawa_IL_LightHawk_June
FracSand-Mine-Unimin-NorthUtica_IL_LightHawk_June
FracSand-Mine-Unimin-NorthUtica_IL_June
FracSand-Mine-Unimin-NorthUtica_IL_June
FracSand-Mine-Unimin-NorthUtica_IL_June
FracSand-Mine-Unimin-NorthUtica_IL_Aug
FracSand-Mine-Unimin-LaSalleCounty_IL_June
FracSand-Mine-Unimin_TechniSand-TroyGrove_IL_May
FracSand-Mine-Unimin_TechniSand-TroyGrove_IL_June
FracSand-Mine-NorthernWhiteSands-Ottawa_IL_May
FracSand-Mine-NorthernWhiteSands-Ottawa_IL_LightHawk_May
FracSand-Mine-NorthernWhiteSands-Ottawa_IL_June
FracSand-Mine-FairmountSantrol-Wedron_IL_May
FracSand-Mine-FairmountSantrol-Wedron_IL_June
FracSand-Mine-SylvanianMinerals-SouthRockwood_MI_July
FracSand-Mine-SylvanianMinerals-SouthRockwood_MI_Aug
FracSand-Mine-SargentSand-Ludington_MI_Sept
FracSand-Mine-SargentSand-Ludington_MI_Oct
FracSand-Mine-SargentSand-Ludington_MI_Oct
FracSand-Mine-RosyMound-Gr

In [52]:
# the culprits 🙉
for rec in matching_records.iloc[:,-1]: print(rec)

FracSand-Transport-GreatNorthernSand-Dovre-WI_
FracSand-Transport-ChippewaSand-CooksValley-WI_
FracSand-Mine-SuperiorSilicaSands-NewAuburn-WI_
FracSand-Mine-ChieftainSand-Dovre-WI_
Infrastructure-InjectionWell-TrisunEnergyServices-Oakville_TX-Nov


### the above list matches the matches the occurrence dict of 4 empty string and 1 'TX-Nov'
#### We are expecting a trailing underscore then month or season. Lets make special cases out of these two scenarios when `split('_')[-1]` returns either the empty string or 'TX-Nov'....BBUUTTTT, lets just given them to Ted to fix since there are only 5! 
## Note, we will be removing 5 records at this step


In [53]:
df1.shape

(2715, 13)

In [54]:
matching_records.shape

(5, 13)

In [55]:
dd = df1.drop(muddy)
dd.shape

(2710, 13)

In [56]:
ddd = df1.drop(matching_records.index)
ddd.shape

(2710, 13)

In [57]:
filter_me = df1.index.isin( matching_records.index )
df2 = df1[~filter_me]
df2.shape

(2710, 13)

In [58]:
for link in matching_records['URL']: print(link)

https://www.flickr.com/photos/fractracker/49728093488/in/album-72157713730887622
https://www.flickr.com/photos/fractracker/49728093538/in/album-72157713730887622
https://www.flickr.com/photos/fractracker/49728600276/in/album-72157713730887622
https://www.flickr.com/photos/fractracker/49728601571/in/album-72157713730887622
https://www.flickr.com/photos/fractracker/49785865077/in/album-72157715839488573


In [59]:
df2.columns

Index(['PhotoID', 'AlbumTitle', 'Title', 'Date_taken', 'Description', 'URL',
       'Latitude', 'Longitude', 'Title_photo_attr', 'title_split',
       'Title_no_photographer', 'Title_yr', 'Title_no_photographer_or_yr'],
      dtype='object')

In [60]:
for rec in df2['Title_no_photographer_or_yr']: print(rec)

FracSand-Mine-WisconsinProppants-Hixton_WI_June
FracSand-Mine-USSilica-Ottawa_IL_LightHawk_May
FracSand-Mine-USSilica-Ottawa_IL_LightHawk_June
FracSand-Mine-Unimin-NorthUtica_IL_LightHawk_June
FracSand-Mine-Unimin-NorthUtica_IL_June
FracSand-Mine-Unimin-NorthUtica_IL_June
FracSand-Mine-Unimin-NorthUtica_IL_June
FracSand-Mine-Unimin-NorthUtica_IL_Aug
FracSand-Mine-Unimin-LaSalleCounty_IL_June
FracSand-Mine-Unimin_TechniSand-TroyGrove_IL_May
FracSand-Mine-Unimin_TechniSand-TroyGrove_IL_June
FracSand-Mine-NorthernWhiteSands-Ottawa_IL_May
FracSand-Mine-NorthernWhiteSands-Ottawa_IL_LightHawk_May
FracSand-Mine-NorthernWhiteSands-Ottawa_IL_June
FracSand-Mine-FairmountSantrol-Wedron_IL_May
FracSand-Mine-FairmountSantrol-Wedron_IL_June
FracSand-Mine-SylvanianMinerals-SouthRockwood_MI_July
FracSand-Mine-SylvanianMinerals-SouthRockwood_MI_Aug
FracSand-Mine-SargentSand-Ludington_MI_Sept
FracSand-Mine-SargentSand-Ludington_MI_Oct
FracSand-Mine-SargentSand-Ludington_MI_Oct
FracSand-Mine-RosyMound-Gr

In [61]:
# DOUBLE check occurrences of each month/season
month_season_d = dict()
for i in df2.loc[:, 'Title_no_photographer_or_yr'] : 
    ii = i.split('_')[-1]
    if ii in month_season_d: month_season_d[ii] += 1
    else: month_season_d[ii] = 1
month_season_d

{'June': 257,
 'May': 300,
 'Aug': 212,
 'July': 166,
 'Sept': 291,
 'Oct': 118,
 'Nov': 213,
 'Fall': 17,
 'Apr': 19,
 'Dec': 21,
 'Jan': 28,
 'March': 140,
 'November': 1,
 'April': 177,
 'Feb': 41,
 'August': 393,
 'October': 87,
 'Jun': 32,
 'December': 144,
 'Summer': 4,
 'February': 49}

### After removing those 5 rows and double checking the `month_season_d` again, it is safe to extract the last element in split by underscore

In [62]:
# saving month/season in a new column
mon_sea = df2.loc[:, 'Title_no_photographer_or_yr'].map(lambda x: x.split('_')[-1]).copy(deep=True) 
df2['month_season'] = mon_sea
df2

/var/folders/ck/r835p07d6wn30nm2crz2l63c0000gn/T/ipykernel_52154/1027835478.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['month_season'] = mon_sea


,PhotoID,AlbumTitle,Title,Date_taken,Description,URL,Latitude,Longitude,Title_photo_attr,title_split,Title_no_photographer,Title_yr,Title_no_photographer_or_yr,month_season
0,49727911618,"['Frac Sand Mining', 'Wisconsin']",TAuch_FracSand-Mine-WisconsinProppants-Hixton_...,2019-06-05 14:07:05,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/4972...,44.376922,-91.069461,TAuch,"[TAuch, FracSand-Mine-WisconsinProppants-Hixto...",FracSand-Mine-WisconsinProppants-Hixton_WI_Jun...,2019,FracSand-Mine-WisconsinProppants-Hixton_WI_June,June
1,49727914298,"['Frac Sand Mining', 'Illinois', 'Impoundment ...",TAuch_FracSand-Mine-USSilica-Ottawa_IL_LightHa...,2018-05-24 09:19:50,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/4972...,41.349366,-88.865747,TAuch,"[TAuch, FracSand-Mine-USSilica-Ottawa, IL, Lig...",FracSand-Mine-USSilica-Ottawa_IL_LightHawk_May...,2018,FracSand-Mine-USSilica-Ottawa_IL_LightHawk_May,May
2,49727914638,"['Frac Sand Mining', 'Frac Sand Transportation...",TAuch_FracSand-Mine-USSilica-Ottawa_IL_LightHa...,2016-06-24 10:47:44,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/4972...,41.349366,-88.865747,TAuch,"[TAuch, FracSand-Mine-USSilica-Ottawa, IL, Lig...",FracSand-Mine-USSilica-Ottawa_IL_LightHawk_Jun...,2016,FracSand-Mine-USSilica-Ottawa_IL_LightHawk_June,June
3,49727915018,"['Frac Sand Mining', 'Illinois', 'Impoundment ...",TAuch_FracSand-Mine-Unimin-NorthUtica_IL_Light...,2016-06-24 10:36:01,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/4972...,41.339217,-89.001553,TAuch,"[TAuch, FracSand-Mine-Unimin-NorthUtica, IL, L...",FracSand-Mine-Unimin-NorthUtica_IL_LightHawk_J...,2016,FracSand-Mine-Unimin-NorthUtica_IL_LightHawk_June,June
4,49727915083,"['Frac Sand Mining', 'Illinois', 'Impoundment ...",TAuch_FracSand-Mine-Unimin-NorthUtica_IL_June2016,2016-06-24 12:13:10,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/4972...,41.351629,-88.992208,TAuch,"[TAuch, FracSand-Mine-Unimin-NorthUtica, IL, J...",FracSand-Mine-Unimin-NorthUtica_IL_June2016,2016,FracSand-Mine-Unimin-NorthUtica_IL_June,June
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2713,53706158340,"['Communities', 'Culture & Livelihoods', 'Envi...",TAuch_Infrastructure-FracSand_RailSpur-Chiefta...,2024-05-03 15:37:18,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/5370...,45.257286,-91.609020,TAuch,"[TAuch, Infrastructure-FracSand, RailSpur-Chie...",Infrastructure-FracSand_RailSpur-ChieftainSand...,2024,Infrastructure-FracSand_RailSpur-ChieftainSand...,May
2714,53706158375,"['Communities', 'Culture & Livelihoods', 'Envi...",TAuch_Infrastructure-FracSand_RailSpur-Norther...,2024-05-03 15:24:13,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/5370...,45.281691,-91.626303,TAuch,"[TAuch, Infrastructure-FracSand, RailSpur-Nort...",Infrastructure-FracSand_RailSpur-NorthernIndus...,2024,Infrastructure-FracSand_RailSpur-NorthernIndus...,May
2715,53706158545,"['Communities', 'Culture & Livelihoods', 'Envi...",TAuch_Infrastructure-FracSand_Mine-HiCrush-Tre...,2024-05-05 11:05:59,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/5370...,44.347402,-91.360048,TAuch,"[TAuch, Infrastructure-FracSand, Mine-HiCrush-...",Infrastructure-FracSand_Mine-HiCrush-Trempeale...,2024,Infrastructure-FracSand_Mine-HiCrush-Trempeale...,May
2716,53706158630,"['Communities', 'Culture & Livelihoods', 'Envi...",TAuch_Infrastructure-FracSand_Mine-HiCrush-Tre...,2024-05-05 11:03:04,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/5370...,44.354636,-91.362434,TAuch,"[TAuch, Infrastructure-FracSand, Mine-HiCrush-...",Infrastructure-FracSand_Mine-HiCrush-Trempeale...,2024,Infrastruc

In [63]:
# demonstrate
s = 'FracSand-Mine-WisconsinProppants-Hixton_WI_June'
''.join(s.split('_')[:-1])

'FracSand-Mine-WisconsinProppants-HixtonWI'

In [64]:
# removing month/season from the Title_no_photographer_or_yr column
everythin_but_mon_sea = df2.loc[:, 'Title_no_photographer_or_yr'].map(lambda x: ''.join(x.split('_')[:-1])).copy(deep=True) 
df2['Title_N_pic_yr_mon'] = everythin_but_mon_sea
df2

/var/folders/ck/r835p07d6wn30nm2crz2l63c0000gn/T/ipykernel_52154/1529118646.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Title_N_pic_yr_mon'] = everythin_but_mon_sea


,PhotoID,AlbumTitle,Title,Date_taken,Description,URL,Latitude,Longitude,Title_photo_attr,title_split,Title_no_photographer,Title_yr,Title_no_photographer_or_yr,month_season,Title_N_pic_yr_mon
0,49727911618,"['Frac Sand Mining', 'Wisconsin']",TAuch_FracSand-Mine-WisconsinProppants-Hixton_...,2019-06-05 14:07:05,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/4972...,44.376922,-91.069461,TAuch,"[TAuch, FracSand-Mine-WisconsinProppants-Hixto...",FracSand-Mine-WisconsinProppants-Hixton_WI_Jun...,2019,FracSand-Mine-WisconsinProppants-Hixton_WI_June,June,FracSand-Mine-WisconsinProppants-HixtonWI
1,49727914298,"['Frac Sand Mining', 'Illinois', 'Impoundment ...",TAuch_FracSand-Mine-USSilica-Ottawa_IL_LightHa...,2018-05-24 09:19:50,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/4972...,41.349366,-88.865747,TAuch,"[TAuch, FracSand-Mine-USSilica-Ottawa, IL, Lig...",FracSand-Mine-USSilica-Ottawa_IL_LightHawk_May...,2018,FracSand-Mine-USSilica-Ottawa_IL_LightHawk_May,May,FracSand-Mine-USSilica-OttawaILLightHawk
2,49727914638,"['Frac Sand Mining', 'Frac Sand Transportation...",TAuch_FracSand-Mine-USSilica-Ottawa_IL_LightHa...,2016-06-24 10:47:44,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/4972...,41.349366,-88.865747,TAuch,"[TAuch, FracSand-Mine-USSilica-Ottawa, IL, Lig...",FracSand-Mine-USSilica-Ottawa_IL_LightHawk_Jun...,2016,FracSand-Mine-USSilica-Ottawa_IL_LightHawk_June,June,FracSand-Mine-USSilica-OttawaILLightHawk
3,49727915018,"['Frac Sand Mining', 'Illinois', 'Impoundment ...",TAuch_FracSand-Mine-Unimin-NorthUtica_IL_Light...,2016-06-24 10:36:01,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/4972...,41.339217,-89.001553,TAuch,"[TAuch, FracSand-Mine-Unimin-NorthUtica, IL, L...",FracSand-Mine-Unimin-NorthUtica_IL_LightHawk_J...,2016,FracSand-Mine-Unimin-NorthUtica_IL_LightHawk_June,June,FracSand-Mine-Unimin-NorthUticaILLightHawk
4,49727915083,"['Frac Sand Mining', 'Illinois', 'Impoundment ...",TAuch_FracSand-Mine-Unimin-NorthUtica_IL_June2016,2016-06-24 12:13:10,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/4972...,41.351629,-88.992208,TAuch,"[TAuch, FracSand-Mine-Unimin-NorthUtica, IL, J...",FracSand-Mine-Unimin-NorthUtica_IL_June2016,2016,FracSand-Mine-Unimin-NorthUtica_IL_June,June,FracSand-Mine-Unimin-NorthUticaIL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2713,53706158340,"['Communities', 'Culture & Livelihoods', 'Envi...",TAuch_Infrastructure-FracSand_RailSpur-Chiefta...,2024-05-03 15:37:18,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/5370...,45.257286,-91.609020,TAuch,"[TAuch, Infrastructure-FracSand, RailSpur-Chie...",Infrastructure-FracSand_RailSpur-ChieftainSand...,2024,Infrastructure-FracSand_RailSpur-ChieftainSand...,May,Infrastructure-FracSandRailSpur-ChieftainSands...
2714,53706158375,"['Communities', 'Culture & Livelihoods', 'Envi...",TAuch_Infrastructure-FracSand_RailSpur-Norther...,2024-05-03 15:24:13,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/5370...,45.281691,-91.626303,TAuch,"[TAuch, Infrastructure-FracSand, RailSpur-Nort...",Infrastructure-FracSand_RailSpur-NorthernIndus...,2024,Infrastructure-FracSand_RailSpur-NorthernIndus...,May,Infrastructure-FracSandRailSpur-NorthernIndust...
2715,53706158545,"['Communities', 'Culture & Livelihoods', 'Envi...",TAuch_Infrastructure-FracSand_Mine-HiCrush-Tre...,2024-05-05 11:05:59,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/5370...,44.347402,-91.360048,TAuch,"[TAuch, Infrastructure-FracSand, Mine-HiCrush-...",Infrastructure-FracSand_Mine-HiCrush-Trempeale...,2024,Infrastructure-FracSand_Mine-HiCrush-Trempeale...,May,Infrastructure-FracSandMine-HiCrush-Trempealea...
2716,537061

In [65]:
for rec in df2['Title_N_pic_yr_mon']: print(rec)   

FracSand-Mine-WisconsinProppants-HixtonWI
FracSand-Mine-USSilica-OttawaILLightHawk
FracSand-Mine-USSilica-OttawaILLightHawk
FracSand-Mine-Unimin-NorthUticaILLightHawk
FracSand-Mine-Unimin-NorthUticaIL
FracSand-Mine-Unimin-NorthUticaIL
FracSand-Mine-Unimin-NorthUticaIL
FracSand-Mine-Unimin-NorthUticaIL
FracSand-Mine-Unimin-LaSalleCountyIL
FracSand-Mine-UniminTechniSand-TroyGroveIL
FracSand-Mine-UniminTechniSand-TroyGroveIL
FracSand-Mine-NorthernWhiteSands-OttawaIL
FracSand-Mine-NorthernWhiteSands-OttawaILLightHawk
FracSand-Mine-NorthernWhiteSands-OttawaIL
FracSand-Mine-FairmountSantrol-WedronIL
FracSand-Mine-FairmountSantrol-WedronIL
FracSand-Mine-SylvanianMinerals-SouthRockwoodMI
FracSand-Mine-SylvanianMinerals-SouthRockwoodMI
FracSand-Mine-SargentSand-LudingtonMI
FracSand-Mine-SargentSand-LudingtonMI
FracSand-Mine-SargentSand-LudingtonMI
FracSand-Mine-RosyMound-GrandHavenMI
FracSand-Mine-RosyMound-GrandHavenMI
FracSand-Mine-RosyMound-GrandHavenMI
FracSand-Mine-RosyMound-GrandHavenMI
F

In [66]:
st = dict()
for i in df2.loc[:, 'Title_N_pic_yr_mon'] : 
    ii = i.split('-')[-1]
    if ii in st: st[ii] += 1
    else: st[ii] = 1    
len(st)

250

### 250 items is just on the limit of what is manageable to inspect manually. 
#### Things to Notice:
- `Lighthawk` [lighthawk.org](https://www.lighthawk.org/) conservationist aviation company. A type of photo attribution and information we want to keep. 
- if we were to remove the last element of a split by hyphen, in some cases we would removing just state abbreviation, state abbreviation plus county, lighthawk plus state abbreviation, most likely some other edge case(s) not yet found.

In [67]:
st

{'HixtonWI': 15,
 'OttawaILLightHawk': 10,
 'NorthUticaILLightHawk': 4,
 'NorthUticaIL': 7,
 'LaSalleCountyIL': 5,
 'TroyGroveIL': 6,
 'OttawaIL': 4,
 'WedronIL': 14,
 'SouthRockwoodMI': 27,
 'LudingtonMI': 19,
 'GrandHavenMI': 12,
 'MN': 43,
 'TX': 415,
 'CurranWI': 2,
 'WI': 72,
 'AugustaWI': 34,
 'CooksValleyWI': 10,
 'ArlandWI': 11,
 'ChippewaFallsWI': 5,
 'DovreWI': 13,
 'UticaIL': 1,
 'WedronILLightHawk': 3,
 'TXLightHawk': 8,
 'JacksonMonroeCountiesWI': 1,
 'RockwoodMI': 14,
 'ChicagoIL': 1,
 'AlmaCenterWI': 1,
 'UticaILLightHawk': 1,
 'MuskegonMI': 2,
 'MonroeCountyWI': 4,
 'AuburnWI': 20,
 'MississippiRiverIA': 1,
 'LightHawk': 44,
 'GalvestonBayTXLightHawk': 1,
 'PineCreekWatershedPALightHawk': 63,
 'KalkaskaCountyMI': 24,
 'SenecaCountyOH': 5,
 'WarrenCountyOH': 6,
 'CarrollCountyOH': 7,
 'WVSouthWings': 2,
 'DoddridgeCountyWVSouthWings': 1,
 'CorpusChristiTXLightHawk': 19,
 'TiogaCountyPALightHawk': 11,
 'DetroitMI': 12,
 'QuakerCityOHLightHawk': 2,
 'SherwoodWVSouthWings':

In [68]:
# light hawk is a photo attribution. let create a new column for it
df2['LightHawk'] = False
# all(df2['LightHawk'])
all(df2['LightHawk'])

/var/folders/ck/r835p07d6wn30nm2crz2l63c0000gn/T/ipykernel_52154/1168747894.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['LightHawk'] = False


False

In [69]:
# understand `.iterrows()``
hard_indx=0
for df_index, row in df2.iterrows():
    if df_index != hard_indx: 
        print(f"hard coded index mismatch {df_index=} {hard_indx=} ")
        # break
        hard_indx+=1 # b/c index mismatch increment one more time
    hard_indx+=1
# below correspond to the removed data


hard coded index mismatch df_index=53 hard_indx=51 
hard coded index mismatch df_index=54 hard_indx=53 
hard coded index mismatch df_index=101 hard_indx=100 
hard coded index mismatch df_index=103 hard_indx=102 
hard coded index mismatch df_index=615 hard_indx=614 
hard coded index mismatch df_index=2589 hard_indx=2588 
hard coded index mismatch df_index=2593 hard_indx=2592 
hard coded index mismatch df_index=2650 hard_indx=2649 


In [70]:
# understand `.iterrows()``
hard_indx=0
for df_index, row in df2.iterrows():
    if df_index != hard_indx: 
        print(f"hard coded index mismatch {df_index=} {hard_indx=} ")
        # break
        hard_indx+=1
    if df_index != hard_indx: 
        print(f"SECOND TRY hard coded index mismatch {df_index=} {hard_indx=} ") # more than one consecutive row was removed 
    hard_indx+=1


hard coded index mismatch df_index=53 hard_indx=51 
SECOND TRY hard coded index mismatch df_index=53 hard_indx=52 
hard coded index mismatch df_index=54 hard_indx=53 
hard coded index mismatch df_index=101 hard_indx=100 
hard coded index mismatch df_index=103 hard_indx=102 
hard coded index mismatch df_index=615 hard_indx=614 
hard coded index mismatch df_index=2589 hard_indx=2588 
hard coded index mismatch df_index=2593 hard_indx=2592 
hard coded index mismatch df_index=2650 hard_indx=2649 


In [71]:
# light_hawk = list() 
# for index, row in df2.iterrows():
#     i = row['Title_N_pic_yr_mon']
#     ii = i.split('-')[-1]
#     ii = ii.lower()
#     if 'lighthawk' in ii:
#         light_hawk.append(index)
#         # row['LightHawk'] = True # possible  strategy 
#         #df2.loc[df2['Title_N_pic_yr_date']==i, 'LightHawk'] = True # thi will not work because of duplicates
# len(light_hawk) # a great stat for the data report. 
isLightHawk = df2['Title_N_pic_yr_mon'].map(lambda x: 'lighthawk' in x.split('-')[-1].lower()).copy(deep=True)  
df_lh = df2[isLightHawk]
df_lh

,PhotoID,AlbumTitle,Title,Date_taken,Description,URL,Latitude,Longitude,Title_photo_attr,title_split,Title_no_photographer,Title_yr,Title_no_photographer_or_yr,month_season,Title_N_pic_yr_mon,LightHawk
1,49727914298,"['Frac Sand Mining', 'Illinois', 'Impoundment ...",TAuch_FracSand-Mine-USSilica-Ottawa_IL_LightHa...,2018-05-24 09:19:50,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/4972...,41.349366,-88.865747,TAuch,"[TAuch, FracSand-Mine-USSilica-Ottawa, IL, Lig...",FracSand-Mine-USSilica-Ottawa_IL_LightHawk_May...,2018,FracSand-Mine-USSilica-Ottawa_IL_LightHawk_May,May,FracSand-Mine-USSilica-OttawaILLightHawk,False
2,49727914638,"['Frac Sand Mining', 'Frac Sand Transportation...",TAuch_FracSand-Mine-USSilica-Ottawa_IL_LightHa...,2016-06-24 10:47:44,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/4972...,41.349366,-88.865747,TAuch,"[TAuch, FracSand-Mine-USSilica-Ottawa, IL, Lig...",FracSand-Mine-USSilica-Ottawa_IL_LightHawk_Jun...,2016,FracSand-Mine-USSilica-Ottawa_IL_LightHawk_June,June,FracSand-Mine-USSilica-OttawaILLightHawk,False
3,49727915018,"['Frac Sand Mining', 'Illinois', 'Impoundment ...",TAuch_FracSand-Mine-Unimin-NorthUtica_IL_Light...,2016-06-24 10:36:01,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/4972...,41.339217,-89.001553,TAuch,"[TAuch, FracSand-Mine-Unimin-NorthUtica, IL, L...",FracSand-Mine-Unimin-NorthUtica_IL_LightHawk_J...,2016,FracSand-Mine-Unimin-NorthUtica_IL_LightHawk_June,June,FracSand-Mine-Unimin-NorthUticaILLightHawk,False
12,49727921643,"['Frac Sand Mining', 'Illinois']",TAuch_FracSand-Mine-NorthernWhiteSands-Ottawa_...,2018-05-24 09:41:02,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/4972...,41.347433,-88.956320,TAuch,"[TAuch, FracSand-Mine-NorthernWhiteSands-Ottaw...",FracSand-Mine-NorthernWhiteSands-Ottawa_IL_Lig...,2018,FracSand-Mine-NorthernWhiteSands-Ottawa_IL_Lig...,May,FracSand-Mine-NorthernWhiteSands-OttawaILLight...,False
57,49728461041,"['Frac Sand Mining', 'Illinois', 'Impoundment ...",TAuch_FracSand-Mine-USSilica-Ottawa_IL_LightHa...,2018-05-24 09:20:25,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/4972...,41.349800,-88.869888,TAuch,"[TAuch, FracSand-Mine-USSilica-Ottawa, IL, Lig...",FracSand-Mine-USSilica-Ottawa_IL_LightHawk_May...,2018,FracSand-Mine-USSilica-Ottawa_IL_LightHawk_May,May,FracSand-Mine-USSilica-OttawaILLightHawk,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1896,51440429468,"['Coal Mining', 'Freshwater Ecosystems', 'Mich...",TAuch_Infrastructure-Coal-PowerPlant-OntarioPo...,2021-09-03 09:51:22,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/5144...,42.792645,-82.475223,TAuch,"[TAuch, Infrastructure-Coal-PowerPlant-Ontario...",Infrastructure-Coal-PowerPlant-OntarioPowerGen...,2021,Infrastructure-Coal-PowerPlant-OntarioPowerGen...,Sept,Infrastructure-Coal-PowerPlant-OntarioPowerGen...,False
1897,51440429548,"['Coal Mining', 'Freshwater Ecosystems', 'Mich...",TAuch_Infrastructure-Coal-PowerPlant-OntarioPo...,2021-09-03 09:51:14,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/5144...,42.794345,-82.476253,TAuch,"[TAuch, Infrastructure-Coal-PowerPlant-Ontario...",Infrastructure-Coal-PowerPlant-OntarioPowerGen...,2021,Infrastructure-Coal-PowerPlant-OntarioPowerGen...,Sept,Infrastructure-Coal-PowerPlant-OntarioPowerGen...,False
1898,51440429783,"['Explosions, Fires, & Flaring', 'Michigan', '...",TAuch_Infrastructure-OilRefinery-Chemicals-She...,2021-09-03 09:45:38,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/5144...,42.895428,-82.465867,TAuch,"[TAuch, Infrastructure-OilRefinery-Chemicals-S...",Infrastructure-OilRefinery-Chemicals-Shell-StC...,2021,Infrastructure-OilRefinery-C

In [72]:
# at all the indexes of the LightHawk records, set the LightHawk column to True
df2.loc[df_lh.index, 'LightHawk'] = True
df2 

,PhotoID,AlbumTitle,Title,Date_taken,Description,URL,Latitude,Longitude,Title_photo_attr,title_split,Title_no_photographer,Title_yr,Title_no_photographer_or_yr,month_season,Title_N_pic_yr_mon,LightHawk
0,49727911618,"['Frac Sand Mining', 'Wisconsin']",TAuch_FracSand-Mine-WisconsinProppants-Hixton_...,2019-06-05 14:07:05,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/4972...,44.376922,-91.069461,TAuch,"[TAuch, FracSand-Mine-WisconsinProppants-Hixto...",FracSand-Mine-WisconsinProppants-Hixton_WI_Jun...,2019,FracSand-Mine-WisconsinProppants-Hixton_WI_June,June,FracSand-Mine-WisconsinProppants-HixtonWI,False
1,49727914298,"['Frac Sand Mining', 'Illinois', 'Impoundment ...",TAuch_FracSand-Mine-USSilica-Ottawa_IL_LightHa...,2018-05-24 09:19:50,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/4972...,41.349366,-88.865747,TAuch,"[TAuch, FracSand-Mine-USSilica-Ottawa, IL, Lig...",FracSand-Mine-USSilica-Ottawa_IL_LightHawk_May...,2018,FracSand-Mine-USSilica-Ottawa_IL_LightHawk_May,May,FracSand-Mine-USSilica-OttawaILLightHawk,True
2,49727914638,"['Frac Sand Mining', 'Frac Sand Transportation...",TAuch_FracSand-Mine-USSilica-Ottawa_IL_LightHa...,2016-06-24 10:47:44,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/4972...,41.349366,-88.865747,TAuch,"[TAuch, FracSand-Mine-USSilica-Ottawa, IL, Lig...",FracSand-Mine-USSilica-Ottawa_IL_LightHawk_Jun...,2016,FracSand-Mine-USSilica-Ottawa_IL_LightHawk_June,June,FracSand-Mine-USSilica-OttawaILLightHawk,True
3,49727915018,"['Frac Sand Mining', 'Illinois', 'Impoundment ...",TAuch_FracSand-Mine-Unimin-NorthUtica_IL_Light...,2016-06-24 10:36:01,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/4972...,41.339217,-89.001553,TAuch,"[TAuch, FracSand-Mine-Unimin-NorthUtica, IL, L...",FracSand-Mine-Unimin-NorthUtica_IL_LightHawk_J...,2016,FracSand-Mine-Unimin-NorthUtica_IL_LightHawk_June,June,FracSand-Mine-Unimin-NorthUticaILLightHawk,True
4,49727915083,"['Frac Sand Mining', 'Illinois', 'Impoundment ...",TAuch_FracSand-Mine-Unimin-NorthUtica_IL_June2016,2016-06-24 12:13:10,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/4972...,41.351629,-88.992208,TAuch,"[TAuch, FracSand-Mine-Unimin-NorthUtica, IL, J...",FracSand-Mine-Unimin-NorthUtica_IL_June2016,2016,FracSand-Mine-Unimin-NorthUtica_IL_June,June,FracSand-Mine-Unimin-NorthUticaIL,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2713,53706158340,"['Communities', 'Culture & Livelihoods', 'Envi...",TAuch_Infrastructure-FracSand_RailSpur-Chiefta...,2024-05-03 15:37:18,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/5370...,45.257286,-91.609020,TAuch,"[TAuch, Infrastructure-FracSand, RailSpur-Chie...",Infrastructure-FracSand_RailSpur-ChieftainSand...,2024,Infrastructure-FracSand_RailSpur-ChieftainSand...,May,Infrastructure-FracSandRailSpur-ChieftainSands...,False
2714,53706158375,"['Communities', 'Culture & Livelihoods', 'Envi...",TAuch_Infrastructure-FracSand_RailSpur-Norther...,2024-05-03 15:24:13,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/5370...,45.281691,-91.626303,TAuch,"[TAuch, Infrastructure-FracSand, RailSpur-Nort...",Infrastructure-FracSand_RailSpur-NorthernIndus...,2024,Infrastructure-FracSand_RailSpur-NorthernIndus...,May,Infrastructure-FracSandRailSpur-NorthernIndust...,False
2715,53706158545,"['Communities', 'Culture & Livelihoods', 'Envi...",TAuch_Infrastructure-FracSand_Mine-HiCrush-Tre...,2024-05-05 11:05:59,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/5370...,44.347402,-91.360048,TAuch,"[TAuch, Infrastructure-FracSand, Mine-HiCrush-...",Infrastructure-FracSand_Mine-HiCrush-Trempeale...,2024,Infrastructure-FracSand_Mine-HiCrush-Trempeale...,May,Infrastr

### Now that we preserved the lighthawk data in new column lets drop the string 'LightHawk' from the data

In [73]:
def delJustLightHawk(x):
    xx = x.lower()
    if xx.endswith('lighthawk'): return x[:-9]
    return x
no_lighthawk = df2['Title_N_pic_yr_mon'].apply(delJustLightHawk).copy(deep=True)
df2['Title_N_pic_yr_mon_lh'] = no_lighthawk # use the data viewer string search to check the results

/var/folders/ck/r835p07d6wn30nm2crz2l63c0000gn/T/ipykernel_52154/1063390675.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Title_N_pic_yr_mon_lh'] = no_lighthawk # use the data viewer string search to check the results


### Next lets pick off the state abbreviation

In [74]:
st_ab = dict() # will hold the occurrences of the expected data (state abbreviations)
no_st_ab = list() # will hold the unexpected data
for i in df2.loc[:, 'Title_N_pic_yr_mon_lh'] : 
    pat = r"([A-Z]{2})$" #  two capital letters at the end of the string
    match = re.search(pat, i)
    if match: 
        ii = match.group(1)
        if ii in st_ab: st_ab[ii] += 1
        else: st_ab[ii] = 1 
    else: no_st_ab.append(i)
st_ab

{'WI': 356,
 'IL': 81,
 'MI': 328,
 'MN': 43,
 'TX': 484,
 'IA': 1,
 'PA': 468,
 'OH': 644,
 'WV': 36,
 'MD': 3,
 'LA': 87}

In [75]:
no_st_ab, len(no_st_ab)     

(['Ecosystems-Wetlands-GulfCoastTX-',
  'Ecosystems-MustangIsland-CorpusChristiTX-',
  'Ecosystems-MustangIsland-CorpusChristiTX-',
  'Ecosystems-CorpusChristiBayShoals-CorpusChristiTX-',
  'Infrastructure-ProcessingFacility-flaring-Williams-MarshallCounty-WVSouthWings',
  'Infrastructure-CompressorStation-MarkWest-DoddridgeCountyWVSouthWings',
  'Ecosystems-Wetlands-GulfCoastTX-',
  'Ecosystems-CorpusChristiBayShoals-CorpusChristiTX-',
  'Infrastructure-ProcessingFacility-MarkWest-Mobley-WVSouthWings',
  'Infrastructure-ProcessingFacility-MarkWest-SherwoodWVSouthWings',
  'Ecosystems-Wetlands-GulfCoastTX-',
  'Ecosystems-MustangIsland-CorpusChristiTX-',
  'Ecosystems-CorpusChristiBayShoals-CorpusChristiTX-',
  'Ecosystems-CorpusChristiBayShoals-CorpusChristiTX-',
  'Ecosystems-CorpusChristiBayShoals-BaysideTX-',
  'Ecosystems-CorpusChristiBayShoals-CorpusChristiTX-',
  'Infrastructure-ProcessingFacility-Williams-WetzelCountyWVSouthWings',
  'Infrastructure-PipelineExplosion-ETCNorthea

#### Things to notice from the `no_st_sb` `list` 
- Need to just remove a hyphen then try again to to remove `[A-Z]{2}$`
- There are some records with state abbreviation before the county name

In [76]:
# similar set up as the previous cell
st_ab2 = dict()
no_st_ab2 = list()
for i in df2.loc[:, 'Title_N_pic_yr_mon_lh'] : 
    pat = r"([A-Z]{2})$"
    match = re.search(pat, i)
    if match: 
        ii = match.group(1)
        if ii in st_ab2: st_ab2[ii] += 1
        else: st_ab2[ii] = 1 
    elif i.endswith('-'):
        ix = i[:-1] # remove the hyphen
        match = re.search(pat, ix)
        if match:  # try again to look for a state abbreviation
            ii = match.group(1)
            if ii in st_ab2: st_ab2[ii] += 1
            else: st_ab2[ii] = 1 
        else : no_st_ab2.append(i)
    else : no_st_ab2.append(i)

st_ab2

{'WI': 356,
 'IL': 81,
 'MI': 330,
 'MN': 43,
 'TX': 524,
 'IA': 1,
 'PA': 473,
 'OH': 644,
 'WV': 36,
 'MD': 3,
 'LA': 87}

In [77]:
no_st_ab2, len(no_st_ab2)

(['Infrastructure-ProcessingFacility-flaring-Williams-MarshallCounty-WVSouthWings',
  'Infrastructure-CompressorStation-MarkWest-DoddridgeCountyWVSouthWings',
  'Infrastructure-ProcessingFacility-MarkWest-Mobley-WVSouthWings',
  'Infrastructure-ProcessingFacility-MarkWest-SherwoodWVSouthWings',
  'Infrastructure-ProcessingFacility-Williams-WetzelCountyWVSouthWings',
  'Mining-MapleCreekCoalMine-OhioValleyCoal-NobleCountyOHSouthwings',
  'Mining-MapleCreekCoalMine-OhioValleyCoal-AlledoniaOHSouthwings',
  'Infrastructure-CrackerPlantConstruction-Shell-PABeaverCounty',
  'Infrastructure-CrackerPlantConstruction-Shell-PABeaverCounty',
  'Infrastructure-CrackerPlantConstruction-Shell-PABeaverCounty',
  'Infrastructure-RadioactiveFrackingWasteRecycling-AustinMasters-OHBelmontCounty',
  'Infrastructure-RadioactiveFrackingWasteRecycling-AustinMasters-OHBelmontCounty',
  'Infrastructure-RadioactiveFrackingWasteRecycling-AustinMasters-OHBelmontCounty',
  'Infrastructure-RadioactiveFrackingWasteR

### Lets Create a new column with no trailing hyphens or state abbreviations 

In [78]:
def delStateAndHyphen(x):
    pat = r"([A-Z]{2})$"
    match = re.search(pat, x)
    if match: 
        if x[-3] == '-': return x[:-3] # removing a state abbreviation and a hyphen 
        return x[:-2] # removing a state abbreviation
    elif x.endswith('-'):
        x = x[:-1]  # removing a hyphen
        match = re.search(pat, x)
        if match: return x[:-2] # removing a state abbreviation
        return x # already removed a hyphen
    return x # no change

no_st_or_hy = df2.loc[:, 'Title_N_pic_yr_mon_lh'].apply(delStateAndHyphen).copy(deep=True)
df2['Title_N_pc_yr_mon_lh_st']  = no_st_or_hy

/var/folders/ck/r835p07d6wn30nm2crz2l63c0000gn/T/ipykernel_52154/1173365378.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Title_N_pc_yr_mon_lh_st']  = no_st_or_hy


### Word Segmenting Time
`Infrastructure-CompressorStationNEXUSPipeline-DTEEnbridge-Seville` --> ['Infrastructure', 'Compressor', 'Station', 'NEXUS', 'Pipeline', 'DTEE', 'nbridge', Seville']
### Now realizing there is no benefit of this.... idea abandoned!

### Make Mission column with simple split and take first element

In [79]:
x='InfrastructurePowerStations'
ret=''
for idx,let in enumerate(x):
    if let.isupper() and idx>0:
        ret+=' '
        ret+=let
    else: ret+=let
ret

'Infrastructure Power Stations'

In [80]:
def mkMission(x):
    x = x.split('-')[0]
    ret=''
    for idx,let in enumerate(x):
        if let.isupper() and idx>0:
            ret+=' '
            ret+=let
        else: ret+=let
    # capitalize the first letter
    if ret[0].islower(): ret = ret[0].upper() + ret[1:]
    return ret
        
mission =  df2.loc[:, 'Title_N_pc_yr_mon_lh_st'].map(mkMission).copy(deep=True)
df2['Mission'] = mission    
df2

/var/folders/ck/r835p07d6wn30nm2crz2l63c0000gn/T/ipykernel_52154/754658654.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Mission'] = mission


,PhotoID,AlbumTitle,Title,Date_taken,Description,URL,Latitude,Longitude,Title_photo_attr,title_split,Title_no_photographer,Title_yr,Title_no_photographer_or_yr,month_season,Title_N_pic_yr_mon,LightHawk,Title_N_pic_yr_mon_lh,Title_N_pc_yr_mon_lh_st,Mission
0,49727911618,"['Frac Sand Mining', 'Wisconsin']",TAuch_FracSand-Mine-WisconsinProppants-Hixton_...,2019-06-05 14:07:05,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/4972...,44.376922,-91.069461,TAuch,"[TAuch, FracSand-Mine-WisconsinProppants-Hixto...",FracSand-Mine-WisconsinProppants-Hixton_WI_Jun...,2019,FracSand-Mine-WisconsinProppants-Hixton_WI_June,June,FracSand-Mine-WisconsinProppants-HixtonWI,False,FracSand-Mine-WisconsinProppants-HixtonWI,FracSand-Mine-WisconsinProppants-Hixton,Frac Sand
1,49727914298,"['Frac Sand Mining', 'Illinois', 'Impoundment ...",TAuch_FracSand-Mine-USSilica-Ottawa_IL_LightHa...,2018-05-24 09:19:50,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/4972...,41.349366,-88.865747,TAuch,"[TAuch, FracSand-Mine-USSilica-Ottawa, IL, Lig...",FracSand-Mine-USSilica-Ottawa_IL_LightHawk_May...,2018,FracSand-Mine-USSilica-Ottawa_IL_LightHawk_May,May,FracSand-Mine-USSilica-OttawaILLightHawk,True,FracSand-Mine-USSilica-OttawaIL,FracSand-Mine-USSilica-Ottawa,Frac Sand
2,49727914638,"['Frac Sand Mining', 'Frac Sand Transportation...",TAuch_FracSand-Mine-USSilica-Ottawa_IL_LightHa...,2016-06-24 10:47:44,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/4972...,41.349366,-88.865747,TAuch,"[TAuch, FracSand-Mine-USSilica-Ottawa, IL, Lig...",FracSand-Mine-USSilica-Ottawa_IL_LightHawk_Jun...,2016,FracSand-Mine-USSilica-Ottawa_IL_LightHawk_June,June,FracSand-Mine-USSilica-OttawaILLightHawk,True,FracSand-Mine-USSilica-OttawaIL,FracSand-Mine-USSilica-Ottawa,Frac Sand
3,49727915018,"['Frac Sand Mining', 'Illinois', 'Impoundment ...",TAuch_FracSand-Mine-Unimin-NorthUtica_IL_Light...,2016-06-24 10:36:01,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/4972...,41.339217,-89.001553,TAuch,"[TAuch, FracSand-Mine-Unimin-NorthUtica, IL, L...",FracSand-Mine-Unimin-NorthUtica_IL_LightHawk_J...,2016,FracSand-Mine-Unimin-NorthUtica_IL_LightHawk_June,June,FracSand-Mine-Unimin-NorthUticaILLightHawk,True,FracSand-Mine-Unimin-NorthUticaIL,FracSand-Mine-Unimin-NorthUtica,Frac Sand
4,49727915083,"['Frac Sand Mining', 'Illinois', 'Impoundment ...",TAuch_FracSand-Mine-Unimin-NorthUtica_IL_June2016,2016-06-24 12:13:10,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/4972...,41.351629,-88.992208,TAuch,"[TAuch, FracSand-Mine-Unimin-NorthUtica, IL, J...",FracSand-Mine-Unimin-NorthUtica_IL_June2016,2016,FracSand-Mine-Unimin-NorthUtica_IL_June,June,FracSand-Mine-Unimin-NorthUticaIL,False,FracSand-Mine-Unimin-NorthUticaIL,FracSand-Mine-Unimin-NorthUtica,Frac Sand
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2713,53706158340,"['Communities', 'Culture & Livelihoods', 'Envi...",TAuch_Infrastructure-FracSand_RailSpur-Chiefta...,2024-05-03 15:37:18,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/5370...,45.257286,-91.609020,TAuch,"[TAuch, Infrastructure-FracSand, RailSpur-Chie...",Infrastructure-FracSand_RailSpur-ChieftainSand...,2024,Infrastructure-FracSand_RailSpur-ChieftainSand...,May,Infrastructure-FracSandRailSpur-ChieftainSands...,False,Infrastructure-FracSandRailSpur-ChieftainSands...,Infrastructure-FracSandRailSpur-ChieftainSands...,Infrastructure
2714,53706158375,"['Communities', 'Culture & Livelihoods', 'Envi...",TAuch_Infrastructure-FracSand_RailSpur-Norther...,2024-05-03 15:24:13,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/5370...,45.281691,-91.626303,TAuch,"[TAuch, Infrastructure-FracSand, RailSpur-Nort...",Infrastructure-FracSand_RailSpur-

In [81]:
df2['Mission'].value_counts()

Mission
Infrastructure                          1796
Frac Sand                                438
Transportation                           157
Cultural                                 101
Ecosystems                                67
Plastics                                  41
Wildlife                                  22
Mining                                    19
Community                                 11
Communities                               10
Construction                               8
Infrastructure Flaring                     6
Infrastructure Ship                        5
Infrastructure Refineries                  5
Propaganda                                 4
Renewable Energy                           4
Plastics Chemicals                         4
Infrastructurecompressorstation            3
Infrastructurecryogenicfacility            2
Plastics Refinery Tank Farm Terminal       1
Plastics Refinery Tank Farm                1
Waste Dredged Island                       1
Wa

In [82]:
s='FracSand-Mine-UniminTechniSand-TroyGrove'
s.split('-')[1:]

['Mine', 'UniminTechniSand', 'TroyGrove']

In [83]:
def mkMissionDescript (x):
    return x.split('-')[1:]
mission_descript = df2.loc[:, 'Title_N_pc_yr_mon_lh_st'].map(mkMissionDescript).copy(deep=True)
df2['Mission_Descript'] = mission_descript

/var/folders/ck/r835p07d6wn30nm2crz2l63c0000gn/T/ipykernel_52154/3300785152.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Mission_Descript'] = mission_descript


### Mission Descript Column is currently an array. Lets Try to remove the county. The array is strings, so lets ask the question how many of the last elements have have the word county in it

In [84]:
cc =dict()
for i in df2.loc[:, 'Mission_Descript'] : 
    c = i[-1].lower()
    if 'county' in c:
        if c in cc: cc[c] += 1
        else: cc[c] = 1 
cc, len(cc)

({'lasallecounty': 6,
  'monroecounty': 21,
  'kalkaskacounty': 24,
  'senecacounty': 5,
  'warrencounty': 6,
  'carrollcounty': 22,
  'doddridgecountywvsouthwings': 1,
  'tiogacounty': 12,
  'muskegoncounty': 7,
  'marioncounty': 1,
  'wetzelcountywvsouthwings': 1,
  'marshallcounty': 8,
  'lycomingcounty': 74,
  'compressorstationimpoundmentpondlycomingcounty': 2,
  'clintoncounty': 10,
  'pottercounty': 56,
  'northumberlandcounty': 18,
  'eriecounty': 10,
  'compressorstationlycomingcounty': 1,
  'compressorstationpipelineimpoundmentpondlycomingcounty': 1,
  'lackawannacounty': 1,
  'cameroncounty': 1,
  'washingtoncounty': 20,
  'barroncounty': 55,
  'jacksoncounty': 3,
  'antrimcounty': 8,
  'crawfordcounty': 5,
  'butlercounty': 5,
  'noblecounty': 10,
  'clarecounty': 3,
  'oaklandcounty': 1,
  'pickawaycounty': 15,
  'morrowcounty': 21,
  'trumbullcounty': 5,
  'starkcounty': 12,
  'otsegocounty': 5,
  'portagecounty': 22,
  'muskingumcounty': 2,
  'montgomerycounty': 2,
  'ho

## Manual look throu the `cc` dict and remove the key:value pairs that we want to keep in out data. Thus, making `cc` our drop list

In [85]:
drop_lst = ['compressorstationimpoundmentpondlycomingcounty','compressorstationlycomingcounty','energytransferpartnerstuscarawascounty',
            'caithnessenergyguernseycounty','belmontsolidscontrolllcharrisoncounty','exxonsabic+sanpetriciocounty']
# This is the list of keys that will be removed from the dictionary...MEANING we will keeep them in the data. These choices were 
# made because that have more information than just county 
for i in drop_lst: del cc[i]    
cc, len(cc)

({'lasallecounty': 6,
  'monroecounty': 21,
  'kalkaskacounty': 24,
  'senecacounty': 5,
  'warrencounty': 6,
  'carrollcounty': 22,
  'doddridgecountywvsouthwings': 1,
  'tiogacounty': 12,
  'muskegoncounty': 7,
  'marioncounty': 1,
  'wetzelcountywvsouthwings': 1,
  'marshallcounty': 8,
  'lycomingcounty': 74,
  'clintoncounty': 10,
  'pottercounty': 56,
  'northumberlandcounty': 18,
  'eriecounty': 10,
  'compressorstationpipelineimpoundmentpondlycomingcounty': 1,
  'lackawannacounty': 1,
  'cameroncounty': 1,
  'washingtoncounty': 20,
  'barroncounty': 55,
  'jacksoncounty': 3,
  'antrimcounty': 8,
  'crawfordcounty': 5,
  'butlercounty': 5,
  'noblecounty': 10,
  'clarecounty': 3,
  'oaklandcounty': 1,
  'pickawaycounty': 15,
  'morrowcounty': 21,
  'trumbullcounty': 5,
  'starkcounty': 12,
  'otsegocounty': 5,
  'portagecounty': 22,
  'muskingumcounty': 2,
  'montgomerycounty': 2,
  'holmescounty': 2,
  'belmontcounty': 35,
  'waynecounty': 70,
  'harrisoncounty': 24,
  ' muskego

In [86]:
# Now we go thru the Mission Descript column and if the the last element is in the dictionary, we will remove it
def delCounty(x):
    if x[-1].lower() in cc: 
        if len(x) > 1: return x[:-1]
    return x
no_county = df2.loc[:, 'Mission_Descript'].apply(delCounty).copy(deep=True)
df2['Mission Description'] = no_county

/var/folders/ck/r835p07d6wn30nm2crz2l63c0000gn/T/ipykernel_52154/2127742507.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Mission Description'] = no_county


In [87]:
# Re format the mission description
def reformatMissionDescript(x):
    s = ''.join(x)
    ret=''
    for idx,let in enumerate(s):
        if let.isupper() and idx>0:
            ret+=' '
            ret+=let
        else: ret+=let
    return ret

ff = df2['Mission Description'].apply(reformatMissionDescript).copy(deep=True)  
df2['Mission Description'] = ff


/var/folders/ck/r835p07d6wn30nm2crz2l63c0000gn/T/ipykernel_52154/3178791662.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Mission Description'] = ff


In [88]:
df2.columns

Index(['PhotoID', 'AlbumTitle', 'Title', 'Date_taken', 'Description', 'URL',
       'Latitude', 'Longitude', 'Title_photo_attr', 'title_split',
       'Title_no_photographer', 'Title_yr', 'Title_no_photographer_or_yr',
       'month_season', 'Title_N_pic_yr_mon', 'LightHawk',
       'Title_N_pic_yr_mon_lh', 'Title_N_pc_yr_mon_lh_st', 'Mission',
       'Mission_Descript', 'Mission Description'],
      dtype='object')

In [89]:
# drop the columns that are no longer needed
df2.drop(columns=['title_split', 'Title_photo_attr', 'Title_no_photographer', 'month_season', 'Title_N_pic_yr_mon',
                  'Title_N_pic_yr_mon_lh', 'Title_N_pc_yr_mon_lh_st', 'Mission_Descript',
                  'Title_yr', 'Title_no_photographer_or_yr'], inplace=True)
df2.columns

/var/folders/ck/r835p07d6wn30nm2crz2l63c0000gn/T/ipykernel_52154/1592017531.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.drop(columns=['title_split', 'Title_photo_attr', 'Title_no_photographer', 'month_season', 'Title_N_pic_yr_mon',


Index(['PhotoID', 'AlbumTitle', 'Title', 'Date_taken', 'Description', 'URL',
       'Latitude', 'Longitude', 'LightHawk', 'Mission', 'Mission Description'],
      dtype='object')

In [93]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [94]:
# %%time
def get_location(latitude, longitude):
    geolocator = Nominatim(user_agent="application")

    reverse = RateLimiter(geolocator.reverse, min_delay_seconds=1)

    location = reverse((latitude, longitude), language='en', exactly_one=True)
    address = location.raw['address']
    state = address.get('state', '')
    county = address.get('county', '')
    return f"{state}|{county}"

# latitude = 41.351629
# longitude =-88.992208	

# state, county, city = get_location(latitude, longitude)
# print("State:", state)
# print("County:", county)

In [ ]:
# literal wall clock time Wall time: 432 ms. NOTE a single run does account for the built rate limiter between calls 
(432/1000*len(df2))/60 

In [ ]:
matching_records.shape

# Don't call again until its threaded!!! too long .... 23min
## make sure both state and county are return or try again...

In [95]:
%%time
get_st_ct = df2.apply(lambda row: get_location(row['Latitude'], row['Longitude']), axis=1).copy(deep=True)
df2['st_count'] = get_st_ct

CPU times: user 1min 10s, sys: 5.04 s, total: 1min 15s
Wall time: 23min 16s


<timed exec>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


# the above is an argument to only process changes to the ! TO do 25ish min of geocoding everytime the script run. this is prone to network error